## Torch Autograd

In [17]:
import torch
from torchvision.models import resnet18, ResNet18_Weights
model = resnet18(weights=ResNet18_Weights.DEFAULT) #Utilisation d'un modèle pré entrainé

data = torch.rand(1, 3, 64, 64) #Ici on obtient une image en couleur de taille 64x64
# Le 1 représente le nombre d'image
# Le 3 représente le nombre de canaux de couleur (RGB)
# La taille de l'image est de 64x64
# Les valeurs sont des variables aléatoires uniformes comprises
labels = torch.rand(1, 1000) #tensor de taille 1000 de variable aléatoire uniforme dans [0,1]

In [18]:
data[0] #Sélection de la 1ère image
data[0][0] #valeur du rouge
data[0][1] #valeur du vert
data[0][2] #valeur du bleu

tensor([[0.7207, 0.1218, 0.7022,  ..., 0.5293, 0.3872, 0.9124],
        [0.3753, 0.0727, 0.8493,  ..., 0.7765, 0.0800, 0.1002],
        [0.7235, 0.9746, 0.4326,  ..., 0.9798, 0.8872, 0.1199],
        ...,
        [0.9210, 0.4000, 0.5745,  ..., 0.7547, 0.9454, 0.6108],
        [0.3263, 0.4572, 0.4528,  ..., 0.5154, 0.1316, 0.2462],
        [0.0651, 0.2648, 0.1849,  ..., 0.2933, 0.8811, 0.0258]])

In [19]:
prediction = model(data)
# Forward pass, on fait passer les données à travers le model pour obtenir une prédiction.

In [20]:
loss = (prediction - labels).sum() #Calcul de la fonction de perte par la somme des différences
lossL1 = (prediction - labels).abs().sum() #Calcul de la fonction de perte par la somme des différences absolue
lossMSE = ((prediction - labels)**2).mean() #Calcul de la fonction de perte par la moyenne des différences au carré

#loss.backward() #Calcul du gradient de la fonction de perte par rapport aux paramètres du modèle
#lossL1.backward() #Calcul du gradient de la fonction de perte par rapport aux paramètres du modèle
lossMSE.backward() #Calcul du gradient de la fonction de perte par rapport aux paramètres du modèle

In [28]:
print(loss)
print(lossL1)
print(lossMSE)

tensor(-489.3982, grad_fn=<SumBackward0>)
tensor(807.4490, grad_fn=<SumBackward0>)
tensor(1.0121, grad_fn=<MeanBackward0>)


In [10]:
#Définition de l'optimiseur
#Ici on utilise l'optimiseur SGD (Descente de Gradient Stochastique)
#lr définit le taux d'apprentissage
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

In [11]:
#On ajuste les paramètre du modèle en appliquant une descente de gradient pour minimiser la fonction de perte
optim.step()

## Differentiation in Autograd

In [37]:
#On crée un tensor a = [2.,3.], requires_grad=True indique que tout les gradients de a doivent être calculé
a = torch.tensor([4., 3.], requires_grad=True)
#On crée un tensor b = [6.,4.], requires_grad=True indique que tout les gradients de b doivent être calculé
b = torch.tensor([6., 4.], requires_grad=True)

In [38]:
Q = 3*a**3 - b**2 # On crée un tensor Q à partir de a et b
Q

tensor([156.,  65.], grad_fn=<SubBackward0>)

In [39]:
external_grad = torch.tensor([1., 1.]) # Point de départ de la rétropropagation
Q.backward(gradient=external_grad) #Calcul du gradient de la fonction de perte par rapport aux paramètres avec requires_grad=True

In [43]:
print(9*a**2)
print(-2*b)

print(a.grad) #Gradient de a
print(b.grad) #Gradient de b

tensor([144.,  81.], grad_fn=<MulBackward0>)
tensor([-12.,  -8.], grad_fn=<MulBackward0>)
tensor([144.,  81.])
tensor([-12.,  -8.])


In [ ]:
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


## Exclusion from the DAG

In [49]:
x = torch.randn(5, 5) #On crée un tensor de taille 5x5 de variable aléatoire normale centrée réduite
y = torch.rand(5, 5) #On crée un tensor de taille 5x5 de variable aléatoire uniforme
z = torch.rand((5, 5), requires_grad=True) #On crée un tensor de taille 5x5 de variable aléatoire uniforme

a = x + y
print(f"Does `a` require gradients?: {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")
c = x + y + z
print(f"Does `c` require gradients?: {c.requires_grad}")

Does `a` require gradients?: False
Does `b` require gradients?: True
Does `c` require gradients?: True


In [50]:
from torch import nn, optim

model = resnet18(weights=ResNet18_Weights.DEFAULT)

#On gèle l'ensemble des paramètres du model, donc lors de l'autograd les gradient ne seront pas calculés sur ces paramètres
for param in model.parameters():
    param.requires_grad = False

In [51]:
model.fc = nn.Linear(512, 10)
#model.fc fait référence à la dernière couche du modèle. Ici on créer un classificateur avec 10 labels.

In [52]:
#Comme tous les paramètre sont gelés sauf le classificateur, l'optimisation ne se fait seulement que sur celle ci.
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)